# Assignment 1-1: Web Scraping

## Objective

Data scientists often need to crawl data from websites and turn the crawled data (HTML pages) to structured data (tables). Thus, web scraping is an essential skill that every data scientist should master. In this assignment, you will learn the followings:


* How to download HTML pages from a website?
* How to extract relevant content from an HTML page? 

Furthermore, you will gain a deeper understanding of the data science lifecycle.

**Requirements:**

1. Please use [pandas.DataFrame](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html) rather than spark.DataFrame to manipulate data.

2. Please use [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/) rather than [lxml](http://lxml.de/) to parse an HTML page and extract data from the page.

3. Please follow the python code style (https://www.python.org/dev/peps/pep-0008/). If TA finds your code hard to read, you will lose points. This requirement will stay for the whole semester.

## Preliminary

If this is your first time to write a web scraper, you need to learn some basic knowledge of this topic. I found that this is a good resource: [Tutorial: Web Scraping and BeautifulSoup](https://www.dataquest.io/blog/web-scraping-beautifulsoup/). 

Please let me know if you find a better resource. I'll share it with the other students.

## Overview

Imagine you are a data scientist working at SFU. Your job is to extract insights from SFU data to answer questions. 

In this assignment, you will do two tasks. Please recall the high-level data science lifecycle below. I suggest that when doing this assignment, please remind yourself of what data you collected and what questions you tried to answer.


<img src="lifecycle.png" width="500">
<center><h4>Figure 1. Data Science Lifecycle</h4></center>

## Task 1: SFU CS Faculty Members

Sometimes you don't know what questions to ask. No worries. Start collecting data first. 

In Task 1, your job is to write a web scraper to extract the faculty information from this page: [http://www.sfu.ca/computing/people/faculty.html](http://www.sfu.ca/computing/people/faculty.html).




### (a) Crawl Web Page

A web page is essentially a file stored in a remote machine (called web server). Please write code to download the HTML page and save it as a text file ("csfaculty.html").

In [ ]:
# write your code
import requests
url = "http://www.sfu.ca/computing/people/faculty.html"
r = requests.get(url, allow_redirects=True)
filename = "csfaculty.html"
open(filename, 'wb').write(r.content)

### (b) Extract Structured Data

Please write code to extract relevant content (name, rank, area, profile, homepage) from "csfaculty.html" and save them as a CSV file (like [faculty_table.csv](./faculty_table.csv)). 

In [ ]:
# write your code 
from bs4 import BeautifulSoup as bs
import pandas as pd
file = open('./csfaculty.html','r',encoding="utf-8")
html = file.read()
bs = bs(html,"html.parser")
divs = bs.find_all(lambda tag: tag.name == 'div' and tag.get('class') == ['text'])

name = []
rank = []
area = []
profile = []
homepage = []

for div in divs:
    index = 1

    for child in div.children:
        
        if index == 1: 
            name.append(str(child).split(',')[0].split('>')[-1])
            rank.append(str(child).split(',')[1].split('<')[0][1:])
            pass
        if index == 3:
            index_area = 1
            for string in child.stripped_strings:
                if index_area == 2:
                    if string is None:
                        string = ""
                    area.append(string)
                index_area+=1
        if index == 5:
            atags = child.find_all('a')
            profile_link = ''
            homepage_link = ''
            for atag in atags:
                if atag.get('href'):
                    if str(atag.string).replace(u'\xa0','')=='Profile & Contact Information':
                        profile_link = atag.get('href')
                        if not profile_link.startswith('http://www.sfu.ca'):
                            profile_link = 'http://www.sfu.ca' + profile_link
                    elif atag.string == 'Home Page':
                        homepage_link = atag.get('href')
            profile.append(profile_link)
            homepage.append(homepage_link)
            
        index+=1

area.append('')
profile.append('')
homepage.append('')

faculty = pd.DataFrame({
    'name': name,
    'rank': rank,
    'areas':area,
    'profile': profile,
    'homepage': homepage
})
faculty.to_csv('my_faculty_table.csv', index=False)




### (c) Interesting Finding

Note that you don't need to do anything for Task 1(c). The purpose of this part is to give you some sense about how to leverage exploratory data analysis (EDA) to come up with interesting questions about the data. EDA is an important topic in data science; you will  learn it soon from this course. 


First, please install [dataprep](http://dataprep.ai).
Then, run the cell below. 
It shows a bar chart for every column. What interesting findings can you get from these visualizations? 

In [ ]:
from dataprep.eda import plot
import pandas as pd

df = pd.read_csv("my_faculty_table.csv")
plot(df)


Below are some examples:

**Finding 1:** Professor# (26) is more than 2x larger than Associate Professor# (10). 

**Questions:** Why did it happen? Is it common in all CS schools in Canada? Will the gap go larger or smaller in five years? What actions can be taken to enlarge/shrink the gap?


**Finding 2:** The Homepage has 22% missing values. 

**Questions:** Why are there so many missing values? Is it because many faculty do not have their own homepages or do not add their homepages to the school page? What actions can be taken to avoid this to happen in the future? 

## Task 2: Age Follows Normal Distribution?

### (a) Crawl Web Page

In this task, you start with a question and then figure out what data to collect.

The question that you are interested in is `Does SFU CS faculty age follow a normal distribution?`

To estimate the age of a faculty member, you can collect the year in which s/he graduates from a university (`gradyear`) and then estimate `age` using the following equation:

$$age \approx 2021+23 - gradyear$$

For example, if one graduates from a university in 1990, then the age is estimated as 2021+23-1990 = 54. 



You notice that faculty profile pages contain graduation information. For example, you can see that Dr. Jiannan Wang graduated from Harbin Institute of Technology in 2008 at [http://www.sfu.ca/computing/people/faculty/jiannanwang.html](http://www.sfu.ca/computing/people/faculty/jiannanwang.html). 


Please write code to download the 65 profile pages and save each page as a text file. 

In [ ]:
# Write your code'
df = pd.read_csv('my_faculty_table.csv')

df_profile = df['profile'].dropna()
df_profile
index = 0 
for profile in df_profile:
    r = requests.get(profile, allow_redirects=True)
    filename = 'data/' + str(index) + '_profile.html'
    open(filename, 'wb').write(r.content)
    index += 1


### (b) Extract Structured Data

Please write code to extract the earliest graduation year (e.g., 2008 for Dr. Jiannan Wang) from each profile page, and create a csv file like [faculty_grad_year.csv](./faculty_grad_year.csv). 

In [ ]:
# write your code here
import re
def extract_age(page):
    file = open(page,'r',encoding="utf-8")
    html = file.read()
    bs1 = bs(html,"html.parser")
    name_session = bs1.find('title')
    name = name_session.string.split(' -')[0]
    education_session = bs1.find(lambda tag: tag.get_text().strip() in ['Education', 'Education:'])
    education_content = education_session.find_next()
    if not education_content.contents:
        education_content = education_content.find_next()
    earliest_gradudation = float('inf')
    for i in education_content.contents:
        graduation_year = re.findall('\d+',str(i.string))
        
        if graduation_year:
            graduation_year = map(int, graduation_year)
            curr_min = min(graduation_year)
            earliest_gradudation = min(earliest_gradudation, curr_min)
    if earliest_gradudation == float('inf'):
        earliest_gradudation = None
    return name, earliest_gradudation

names = []
graduation_years = []
for i in range(len(df_profile)):
    name, year =extract_age('./data/' + str(i) + '_profile.html')
    names.append(name)
    graduation_years.append(year)

graduation = pd.DataFrame({
    'name': names,
    'graduation_year': graduation_years,
})
graduation.to_csv('my_faculty_grad_year.csv', index=False)

    

### (c) Interesting Finding

Similar to Task 1(c), you don't need to do anything here. Just look at different visualizations w.r.t. age and give yourself an answer to the question: `Does SFU CS faculty age follow a normal distribution?`

In [ ]:
from dataprep.eda import plot
import pandas as pd

df = pd.read_csv("faculty_grad_year.csv")
df["age"] = 2021+23-df["gradyear"]

plot(df, "age")

## Submission

Complete the code in this notebook, and submit it to the CourSys activity `Assignment 1`.

## Bonus: Contribute to DataPrep

**Total Bonus: 0.2 + 0.3 = 0.5**

1. If you create an issue (i.e., report a bug or request a feature) at [link1](https://github.com/sfu-db/dataprep/issues), you will get **0.2** bonus points.

2.  Creating more issues will *not* give you more bonus points, but you are encouraged to do so for learning more.

2. If you send a pull request (i.e., fix a bug or implement a feature) at [link1](https://github.com/sfu-db/dataprep/pulls) and the pull request gets merged into the repo, you will get **0.3** bonus points. 

4.  Sending more pull requests will *not* give you more bonus points, but you are encouraged to do so for learning more.

5. These bonus points will be directly added to your final grade.






**How to Submit**
* Submit github link(s) to the CourSys activity `Bonus`.
* Due on `March 13, 2022`

If you love dataprep, please support it by simply clicking the **<span style="color:red">Star</span>** on [Github](https://github.com/sfu-db/dataprep).
<img src="dataprep-star-2022.png" width="1000">